In [2]:
import pyvisa
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtWidgets
import sys
import numpy as np
import time

In [5]:

# Connect to the PM100USB
rm = pyvisa.ResourceManager()
instruments = rm.list_resources()
print(instruments)

# Automatically connect to Thorlabs PM100USB
pm100 = None
for resource in instruments:
    if 'USB' in resource:
        try:
            inst = rm.open_resource(resource)
            idn = inst.query("*IDN?")
            if "PM100" in idn:
                pm100 = inst
                print(f"Connected to {idn}")
                break
        except Exception as e:
            print(f"Failed to connect to {resource}: {e}")

if pm100 is None:
    raise Exception("Thorlabs PM100USB not found")

# Setup plot
app = QtWidgets.QApplication(sys.argv)
win = pg.GraphicsLayoutWidget(show=True, title="Real-time Power Meter Reading")
win.resize(800, 400)
plot = win.addPlot(title="Optical Power vs Time")
curve = plot.plot(pen='y')
plot.setLabel('left', "Power", units='W')
plot.setLabel('bottom', "Time", units='s')

data = np.zeros(100)
timestamps = np.linspace(-10, 0, 100)

# Timer callback to update plot
def update():
    global data, timestamps
    try:
        power = float(pm100.query("READ?"))
        data = np.roll(data, -1)
        data[-1] = power
        timestamps = np.roll(timestamps, -1)
        timestamps[-1] = time.time() - start_time
        curve.setData(timestamps - timestamps[-1], data)
    except Exception as e:
        print(f"Read error: {e}")

# Timer
timer = QtCore.QTimer()
timer.timeout.connect(update)
timer.start(100)  # every 100 ms

start_time = time.time()
sys.exit(app.exec_())

('USB0::0x1313::0x8072::P2000053::INSTR', 'ASRL1::INSTR', 'ASRL3::INSTR', 'ASRL10::INSTR')
Connected to Thorlabs,PM100USB,P2000053,1.1.0



SystemExit: 0

c:\Users\My lAb NajA ii\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
